<a href="https://colab.research.google.com/github/HanifFaishalH/pbl_jawara/blob/main/Model_PCVK_Kel3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Mengambil Dataset Batik

In [ ]:
import pandas as pd
import os

# path folder dataset
source_data = '/content/drive/MyDrive/PBL/data_batik'

classes = os.listdir(source_data)

data_info = []

for kelas in classes:
    kelas_path = os.path.join(source_data, kelas)

    # Hitung jumlah file gambar
    jumlah_file = len([f for f in os.listdir(kelas_path)
                       if os.path.isfile(os.path.join(kelas_path, f))])

    data_info.append({
        "Kategori": kelas,
        "Jumlah Gambar": jumlah_file
    })

# Buat DataFrame informasi dataset
df = pd.DataFrame(data_info)

print(df)

  Kategori  Jumlah Gambar
0    Tulis             48
1      Cap             48


In [ ]:
import os
import shutil
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

source_data = '/content/drive/MyDrive/PBL/batik-dataset'
base_augmented_output = '/content/drive/MyDrive/PBL/data_batik_augmented_discrete'
ORIGINAL_IMAGES_PER_CLASS = 48

AUGMENTATION_CONFIGS = [
    {"name": "zoom", "datagen_params": {"zoom_range": [0.5, 1.5], "fill_mode": 'nearest'}},
    {"name": "vflip", "datagen_params": {"vertical_flip": True, "fill_mode": 'nearest'}},
    {"name": "hflip", "datagen_params": {"horizontal_flip": True, "fill_mode": 'nearest'}}
]

if not os.path.exists(base_augmented_output):
    os.makedirs(base_augmented_output)

print(f"Memulai proses augmentasi diskrit dari: {source_data}")

classes = os.listdir(source_data)
total_overall_final = 0

for class_name in classes:
    source_class_path = os.path.join(source_data, class_name)
    target_class_path = os.path.join(base_augmented_output, class_name)

    if not os.path.exists(target_class_path):
        os.makedirs(target_class_path)

    image_files = [f for f in os.listdir(source_class_path)
                   if os.path.isfile(os.path.join(source_class_path, f))
                   and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(f"-> Melewati kelas '{class_name}': Tidak ditemukan gambar.")
        continue

    original_count = 0
    augmented_total_count = 0

    for original_image_name in image_files:
        original_image_path = os.path.join(source_class_path, original_image_name)
        file_prefix = original_image_name.split(".")[0]

        shutil.copy(original_image_path, os.path.join(target_class_path, original_image_name))
        original_count += 1

        try:
            img = Image.open(original_image_path)
            x = np.asarray(img)
            x = x.reshape((1,) + x.shape)

            for aug_config in AUGMENTATION_CONFIGS:
                aug_name = aug_config["name"]
                aug_params = aug_config["datagen_params"]

                aug_output_folder = os.path.join(target_class_path, f'aug_{aug_name}_{class_name}')
                if not os.path.exists(aug_output_folder):
                    os.makedirs(aug_output_folder)

                datagen_discrete = ImageDataGenerator(**aug_params)

                i = 0
                for batch in datagen_discrete.flow(x, batch_size=1,
                                                   save_to_dir=aug_output_folder,
                                                   save_prefix=f'{aug_name}_{file_prefix}',
                                                   save_format='jpeg'):
                    i += 1
                    augmented_total_count += 1
                    if i >= 1:
                        break

        except Exception as e:
            print(f"   - Gagal mengaugmentasi gambar '{original_image_name}' dengan {aug_name}. Error: {e}")

    total_final = original_count + augmented_total_count
    total_overall_final += total_final

    print(f"|--- Kelas '{class_name}' selesai diproses.")
    print(f"|--- Total Gambar Asli (di folder utama): {original_count}")
    print(f"|--- Total Augmentasi yang Dibuat: {augmented_total_count} (48 x 3)")
    print(f"|--- **TOTAL GAMBAR AKHIR PER KELAS: {total_final}**")
    print("-" * 50)


print("\nProses augmentasi diskrit selesai!")
print(f"SETIAP KELAS SEKARANG MENGANDUNG TOTAL {48} ASLI + {48*3} AUGMENTED = {192} GAMBAR.")